# Modelo LTR - Learning To Rank

In [90]:
import dagshub
import mlflow

import polars as pl
import numpy as np

import lightgbm as lgb

from sklearn.model_selection import train_test_split
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

from ml_utils import search_best_model


dagshub.init(repo_owner='abdala9512', repo_name='dsrp-machine-learning-engineering-4', mlflow=True)

Initialized MLflow to track repo "abdala9512/dsrp-machine-learning-engineering-4"

Repository abdala9512/dsrp-machine-learning-engineering-4 initialized!

In [85]:
import numpy as np
import polars as pl


# -----------------------------------------------------------
# Utilidades para métricas de ranking
# -----------------------------------------------------------

def dcg(relevances):
    """Discounted Cumulative Gain."""
    relevances = np.array(relevances)
    discounts = 1 / np.log2(np.arange(2, len(relevances) + 2))
    return np.sum(relevances * discounts)


def ndcg_at_k(true_rels, pred_scores, k=10):
    """Compute NDCG@k for una query individual."""
    idx = np.argsort(-pred_scores)[:k]
    sorted_true = np.array(true_rels)[idx]

    dcg_k = dcg(sorted_true)

    # Ideal DCG (ordenamos relevâncias por orden descendente)
    ideal_sorted_true = np.sort(true_rels)[::-1][:k]
    idcg_k = dcg(ideal_sorted_true)

    return float( dcg_k / idcg_k if idcg_k > 0 else 0.0 )



## 1. Preparación de datos

In [40]:
ltr_df = pl.read_parquet("data/ltr_imdb_dataset.parquet")
ltr_df.head()

query_id,query_text,imdb_id,title,sim_embedding,imdb_rating,imdb_votes_log,year,genres,rel_score,label
i32,str,str,str,f32,f64,f64,i32,str,f64,i32
1,"""best drama movies""","""tt9179430""","""Vikram""",0.463414,8.3,11.37896,2022,"""Action,Crime,Thriller""",0.669529,3
1,"""best drama movies""","""tt18163024""","""Chaaruseela""",0.501941,8.5,7.604396,2022,null,0.642465,3
1,"""best drama movies""","""tt0100998""","""Dreams""",0.483811,7.7,10.370048,1990,"""Drama,Fantasy""",0.640197,3
1,"""best drama movies""","""tt0187231""","""Barking at the Stars""",0.425586,8.6,9.091332,1998,"""Comedy,Romance""",0.635807,3
1,"""best drama movies""","""tt9006564""","""2018""",0.427564,8.3,9.691222,2023,"""Action,Drama,Thriller""",0.632621,3


In [21]:

# columnas feature (elige las que quieras incluir)
feature_cols = [c for c in ltr_df.columns if c.startswith("emb_")] + [
    "sim_embedding",
    "imdb_rating",
    "imdb_votes_log",
]


print("X shape:", X.shape)
print("y shape:", y.shape)
print("num groups:", len(groups))


X shape: (21000, 3)
y shape: (21000,)
num groups: 210


/var/folders/65/xyj4xjq9355355kxr06pqfy80000gn/T/ipykernel_12251/3470404949.py:20: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  .count()


In [74]:
qid_list = ltr_df["query_id"].unique().to_list()

split = int(len(qid_list) * (1 - 0.8))

train_ids = qid_list[:len(qid_list) - split]
test_ids = qid_list[len(qid_list) - split:]

ltr_train = ltr_df.filter(pl.col("query_id").is_in(train_ids))
ltr_test = ltr_df.filter(pl.col("query_id").is_in(test_ids))

X_train = ltr_train.select(feature_cols).to_numpy()
y_train = ltr_train["label"].to_numpy()
train_groups = ltr_train.group_by("query_id").count()["count"].to_numpy()

X_test = ltr_test.select(feature_cols).to_numpy()
y_test = ltr_test["label"].to_numpy()

/var/folders/65/xyj4xjq9355355kxr06pqfy80000gn/T/ipykernel_12251/2807853531.py:13: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  train_groups = ltr_train.group_by("query_id").count()["count"].to_numpy()


In [77]:
search_space = {
    "max_depth": hp.quniform("max_depth", 3, 10, 1),
    "n_estimators": hp.quniform("n_estimators", 50, 500, 50)
}

def objective(params):

    normalized_params = {
        "max_depth": int(params["max_depth"]),
        "n_estimators": int(params["n_estimators"])
    }

    ranker = lgb.LGBMRanker(
        objective="lambdarank",
        boosting_type="gbdt",
        metric="ndcg",
        **normalized_params
    )
    ranker.fit(
        X_train,
        y_train,
        group=train_groups
    )

    predictions = ranker.predict(X_test)
    print(predictions)
    
    ndcg_k = ndcg_at_k(y_test, predictions,k=10)
    print(f"nDCG@10: {ndcg_k}")
    return {
        "loss": -ndcg_k, "status": STATUS_OK
    }

trials = Trials()

BEST_RANKER = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    trials=Trials(),
    max_evals=3,
)

BEST_RANKER

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000300 seconds.                                                                            
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 584                                                                                                                                                   
[LightGBM] [Info] Number of data points in the train set: 16900, number of used features: 3                                                                                        
[ 10.9016742    7.24570503   7.62637673 ... -11.10508294 -11.31218037                                                                                                              
 -11.06717131]
nDCG@10: 1.0                                                                                                                                                                       
[LightGBM] [Info] Auto-choo

/Users/miguelarquezabdala/repos/dsrp-machine-learning-engineering-4/notebooks/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(



[ 5.64615466  5.13419575  5.25469951 ... -5.43571607 -5.43571607                                                                                                                   
 -5.43571607]
nDCG@10: 1.0                                                                                                                                                                       
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000424 seconds.                                                                            
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 584                                                                                                                                                   
[LightGBM] [Info] Number of data points in the train set: 16900, number of used features: 3                                                                                        
 67%|███████████████████████

/Users/miguelarquezabdala/repos/dsrp-machine-learning-engineering-4/notebooks/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                                         
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                                         
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                                         
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                                         
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                                         
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                          

/Users/miguelarquezabdala/repos/dsrp-machine-learning-engineering-4/notebooks/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(



{'max_depth': np.float64(8.0), 'n_estimators': np.float64(200.0)}

In [79]:
best_params = {
    "max_depth": int(BEST_RANKER["max_depth"]),
    "n_estimators": int(BEST_RANKER["n_estimators"])
}

In [82]:
mlflow.create_experiment("LTR - Claser 17 Diciembre")

'1'

In [150]:
mlflow.set_experiment("LTR - Clase 17 Diciembre")
MODEL_NAME = "ltr-dsrpflix-prd"

with mlflow.start_run(run_name="LGBM Ranker -  Optimizado") as run:

    _best_ranker = lgb.LGBMRanker(
            objective="lambdarank",
            boosting_type="gbdt",
            metric="ndcg",
            **best_params
        )
    _best_ranker.fit(
        X_train,
        y_train,
        group=train_groups
    )

    predictions = _best_ranker.predict(X_test)    
    ndcg_5 = ndcg_at_k(y_test, predictions,k=5)
    ndcg_10 = ndcg_at_k(y_test, predictions,k=10)
    ndcg_15 = ndcg_at_k(y_test, predictions,k=15)
    ranker_metrics = {
            "nDCG5": ndcg_5,
            "nDCG10": ndcg_10,
            "nDCG15": ndcg_15,
        }
    
    mlflow.log_params(best_params)
    mlflow.log_metrics(ranker_metrics)
    mlflow.lightgbm.log_model(
        lgb_model=_best_ranker, 
        artifact_path="model",
        input_example=X_train,
        registered_model_name=MODEL_NAME,
    )

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000365 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 584
[LightGBM] [Info] Number of data points in the train set: 16900, number of used features: 3


/Users/miguelarquezabdala/repos/dsrp-machine-learning-engineering-4/notebooks/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
2025/12/17 21:25:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/miguelarquezabdala/repos/dsrp-machine-learning-engineering-4/notebooks/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
2025/12/17 21:25:16 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
/Users/miguelarquezabdala/repos/dsrp-machine-learning-engineering-4/notebooks/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, b

🏃 View run LGBM Ranker -  Optimizado at: https://dagshub.com/abdala9512/dsrp-machine-learning-engineering-4.mlflow/#/experiments/2/runs/656c6f36976e44e3a3717291dd42c496
🧪 View experiment at: https://dagshub.com/abdala9512/dsrp-machine-learning-engineering-4.mlflow/#/experiments/2


In [162]:
METRIC = "nDCG5"

client = mlflow.MlflowClient()
_registered_model = client.get_registered_model(name=MODEL_NAME)
_registered_model

CHAMPION_MODEL_VERSION = _registered_model.aliases["champion"]
CHAMPION_METRIC = mlflow.get_run(run_id=client.get_model_version(name=MODEL_NAME, version=CHAMPION_MODEL_VERSION).run_id).data.metrics[METRIC] 

LATEST_MODEL_VERSION = _registered_model.latest_versions[0].version


if LATEST_MODEL_VERSION == CHAMPION_MODEL_VERSION:
    print("Ultima versión del modelo es Champion")
else:
    CHALLENGER_METRIC = mlflow.get_run(run_id=client.get_model_version(name=MODEL_NAME,version=LATEST_MODEL_VERSION).run_id).data.metrics[METRIC] 
    if CHALLENGER_METRIC >= CHAMPION_METRIC:
        client.set_registered_model_alias(
         name=MODEL_NAME,
         alias="champion",
         version=LATEST_MODEL_VERSION
        )
        print("Ultimo/Challenger modelo promovido a champion")
            

Ultima versión del modelo es Champion


In [163]:
prod_model = mlflow.lightgbm.load_model(f"models:/{MODEL_NAME}@champion")

/Users/miguelarquezabdala/repos/dsrp-machine-learning-engineering-4/notebooks/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [65]:
def test_single_query(qid, model):
    df = ltr_df.filter(pl.col("query_id") == qid)
    Xq = df.select(feature_cols).to_numpy()

    preds = model.predict(Xq)

    df = df.with_columns(pl.Series("ranker_score", preds))
    return df.sort("ranker_score", descending=True).head(10)

test_single_query(3, model=prod_model)


/Users/miguelarquezabdala/repos/dsrp-machine-learning-engineering-4/notebooks/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(


query_id,query_text,imdb_id,title,sim_embedding,imdb_rating,imdb_votes_log,year,genres,rel_score,label,ranker_score
i32,str,str,str,f32,f64,f64,i32,str,f64,i32,f64
3,"""classic drama movies""","""tt0088930""","""Clue""",0.451998,7.3,11.696346,1985,"""Comedy,Crime,Mystery""",0.601407,3,5.524147
3,"""classic drama movies""","""tt9179430""","""Vikram""",0.437424,8.3,11.37896,2022,"""Action,Crime,Thriller""",0.619876,3,5.519233
3,"""classic drama movies""","""tt0105151""","""The Player""",0.454028,7.5,11.17052,1992,"""Comedy,Crime,Drama""",0.60139,3,5.462067
3,"""classic drama movies""","""tt1462764""","""Indiana Jones and the Dial of …",0.442078,6.5,12.333731,2023,"""Action,Adventure,Sci-Fi""",0.58097,3,5.445153
3,"""classic drama movies""","""tt0048281""","""The Ladykillers""",0.434312,7.6,10.42106,1955,"""Comedy,Crime""",0.58451,3,5.092517
3,"""classic drama movies""","""tt0100998""","""Dreams""",0.431395,7.7,10.370048,1990,"""Drama,Fantasy""",0.58537,3,4.752521
3,"""classic drama movies""","""tt0097108""","""The Cook, the Thief, His Wife …",0.431122,7.5,10.699913,1989,"""Crime,Drama""",0.583644,3,4.685198
3,"""classic drama movies""","""tt0808279""","""Funny Games""",0.460056,6.5,11.60111,2007,"""Crime,Drama,Thriller""",0.580163,3,4.667681
3,"""classic drama movies""","""tt0485510""","""Killing the Shadows""",0.465337,7.5,9.557682,2006,"""Comedy,Drama,History""",0.585477,3,4.577074
